In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
pd.options.display.float_format = '{:.2f}'.format

filename = 'mutations.csv'
data = pd.read_csv(filename, index_col=0)
# samples = number of samples
samples = data.shape[0]


In [12]:
def calc_advanced_metrics(TP, FP, TN , FN ):
    advanced_metrics = []
    accuracy = (TP + TN) / (TP + FP + TN + FN) * 100
    sensitivity = TP / (TP + FN) * 100
    specificity = TN / (TN + FP) * 100
    precision = TP / (TP + FP) * 100
    miss_rate = FN / (FN + TP) * 100
    fdr = FP / (FP + TP) * 100
    forr = FN / (FN + TN + 0.00001) * 100

    advanced_metrics.append([accuracy, sensitivity, specificity, precision, miss_rate, fdr, forr])
    return advanced_metrics


def logby2(x):
    return np.log2(x) if x != 0 else 0


def find_tL_tR(mutation, data):
    tL = data[data[mutation] == 1]
    tR = data[data[mutation] == 0]
    return tL, tR


def find_NC_C(data):
    NC = data[data.index.str.startswith('NC')]
    C = data[data.index.str.startswith('C')]
    return NC, C


def HT(PC, PNC):
    return -PC * logby2(PC) - PNC * logby2(PNC)


def find_gain(mutation, data):
        
        noncancerous, cancerous = find_NC_C(data)
        PC = len(cancerous) / samples
        PNC = len(noncancerous) / samples
        #H(t) = -[pC,t log2(pC,t) + pNC,t log2(pNC,t)]
        #H(t) = -[(probability of cancerous samples) * log2(probability of cancerous samples) + (probability of non-cancerous samples) * log2(probability of non-cancerous samples)]
        HT_value = -PC * logby2(PC) - PNC * logby2(PNC)
        L, R = find_tL_tR(mutation, data)
        PL = len(L) / samples
        NCL, CL = find_NC_C(L)
        if len(L) > 0:
            HTL = -((len(CL) / len(L) + 0.000001) * logby2((len(CL) / len(L)) + 0.000001) + (len(NCL) / len(L) + 0.000001) * logby2((len(NCL) / len(L)) + 0.000001))
        else:
            HTL = 0

        PR = len(R) / samples
        NCR, CR = find_NC_C(R)
        if len(R) > 0:
            HTR = -((len(CR) / len(R) + 0.000001) * logby2((len(CR) / len(R)) + 0.000001) + (len(NCR) / len(R) + 0.00001) * logby2((len(NCR) / len(R)) + 0.00001))
        else:
            HTR = 0
        HST_value = (PL * HTL) + (PR * HTR)
        gain = HT_value - HST_value
        return gain


In [13]:
#get bootstrap samples
def getBootstrapSamples(data):
    bootstrapData = pd.DataFrame()
    # go through the number of samples in data
    for i in range(samples):
        # get a random number between 0 and the number of samples
        randomNum = np.random.randint(0, samples) 
        # find the data at the random number index and add it to the bootstrapData
        bootstrapData = pd.concat([bootstrapData, data.loc[[data.index[randomNum]]]])
        # we dont have to worry about duplicates because we are sampling with replacement

    # get the out of bag data
    # if the index of the data is not in the bootstrapData index, then it is out of bag data
    outOfBagData = data[~data.index.isin(bootstrapData.index)]
    
    return bootstrapData, outOfBagData

In [ ]:
def build_tree_for_forest(bootstrapData, outOfBagData):
    selectedFeatures = pd.DataFrame()
    selectedFeaturesL = pd.DataFrame()
    selectedFeaturesR = pd.DataFrame()
    gainList = []
    gianListL = []
    gainListR = []


    # sqrt(num of mutation from og data)
    num = math.sqrt(data.shape[1])
    # convert it to a int bc sqrt returns a float
    num = int(num)

    usedIndecies = []

    # 0 - num
    for i in range(num):
        # get a random number between 0 and num
        randomNum = np.random.randint(0, num)
        # next step is important bc we dont want replacement
        # if the random number is in the usedIndecies list
        if randomNum in usedIndecies:
            # pick a new random number until it is not in the usedIndecies
            while randomNum in usedIndecies:
                randomNum = np.random.randint(0, num)
        # add that number to the usedIndecies list so we can ignore it next time
        usedIndecies.append(randomNum)
        # add the data at the random number index to the selectedFeatures
        selectedFeatures = pd.concat([selectedFeatures, bootstrapData.iloc[:, randomNum]], axis=1)
        # find the gain of the selected mutation
        gainList.append(find_gain(selectedFeatures.columns[i], bootstrapData))

    # find the max gain
    maxGain = max(gainList)
    # find the index of the max gain
    maxGainIndex = gainList.index(maxGain)
    # find the mutation with the max gain
    selectedMutation = selectedFeatures.columns[maxGainIndex]

    # split data into left and right, where left samples HAVE the mutation and right samples DONT have the mutation
    L, R = find_tL_tR(selectedMutation, bootstrapData)

    usedIndeciesL = []

    for i in range(num):
        randomNum = np.random.randint(0, num)
        if randomNum in usedIndeciesL:
            while randomNum in usedIndeciesL:
                randomNum = np.random.randint(0, num)
        usedIndeciesL.append(randomNum)
        selectedFeaturesL = pd.concat([selectedFeaturesL, L.iloc[:, randomNum]], axis=1)
        gianListL.append(find_gain(selectedFeaturesL.columns[i], L))

    #instead of having to find the max gain like below, try just keep the max gain and only replace it if the new gain is greater

    maxGainL = max(gianListL)
    maxGainIndexL = gianListL.index(maxGainL)
    selectedMutationL = selectedFeaturesL.columns[maxGainIndexL]
    # if the selected mutation is the same as the selected mutation from the first tree
    if(selectedMutationL == selectedMutation):
        # drop the selected mutation
        selectedFeaturesL = selectedFeaturesL.drop(selectedMutationL, axis=1)
        # remove the gain of the selected mutation
        gianListL.remove(maxGainL)
        # find the max gain again
        maxGainL = max(gianListL)
        # find the index of the max gain
        maxGainIndexL = gianListL.index(maxGainL)
        # find the mutation with the max gain
        selectedMutationL = selectedFeaturesL.columns[maxGainIndexL]

    A1, A2 = find_tL_tR(selectedMutationL, L)
    CA1 = len(A1[A1.index.str.startswith('C')])
    NCA1 = len(A1[A1.index.str.startswith('NC')])
    CA2 = len(A2[A2.index.str.startswith('C')])
    NCA2 = len(A2[A2.index.str.startswith('NC')])

    if CA1 > NCA1:
        classified_A1 = 'C'
    else:
        classified_A1 = 'NC'
    
    if CA2 > NCA2:
        classified_A2 = 'C'
    else:
        classified_A2 = 'NC'

    usedIndeciesR = []

    for i in range(num):
        randomNum = np.random.randint(0, num)
        if randomNum in usedIndeciesR:
            while randomNum in usedIndeciesR:
                randomNum = np.random.randint(0, num)
        usedIndeciesR.append(randomNum)
        selectedFeaturesR = pd.concat([selectedFeaturesR, R.iloc[:, randomNum]], axis=1)
        gainListR.append(find_gain(selectedFeaturesR.columns[i], R))
    
    maxGainR = max(gainListR)
    maxGainIndexR = gainListR.index(maxGainR)
    selectedMutationR = selectedFeaturesR.columns[maxGainIndexR]
    # if the selected mutation is the same as the selected mutation from the first tree or the second tree
    if(selectedMutationR == selectedMutation or selectedMutationR == selectedMutationL):
        selectedFeaturesR = selectedFeaturesR.drop(selectedMutationR, axis=1)
        gainListR.remove(maxGainR)
        maxGainR = max(gainListR)
        maxGainIndexR = gainListR.index(maxGainR)
        selectedMutationR = selectedFeaturesR.columns[maxGainIndexR]

    B1, B2 = find_tL_tR(selectedMutationR, R)
    CB1 = len(B1[B1.index.str.startswith('C')])
    NCB1 = len(B1[B1.index.str.startswith('NC')])
    CB2 = len(B2[B2.index.str.startswith('C')])
    NCB2 = len(B2[B2.index.str.startswith('NC')])

    if CB1 > NCB1:
        classified_B1 = 'C'
    else:
        classified_B1 = 'NC'
    
    if CB2 > NCB2:
        classified_B2 = 'C'
    else:
        classified_B2 = 'NC'

    return selectedMutation, selectedMutationL, selectedMutationR, classified_A1, classified_A2, classified_B1, classified_B2, len(outOfBagData)


In [15]:
treeChart = pd.DataFrame(columns=['Top Mutation', 'Top Left Mutation', 'Top Right Mutation', 'A1', 'A2', 'B1', 'B2', 'Out of Bag Data Length'])

# build 25 trees
for i in range(25):
    # get the bootstrap samples and out of bag data from the data
    bootstrapData, outOfBagData = getBootstrapSamples(data)
    # build the tree and add it to the treeChart
    treeChart.loc[i] = build_tree_for_forest(bootstrapData, outOfBagData)
    
treeChart.head(25)

,Top Mutation,Top Left Mutation,Top Right Mutation,A1,A2,B1,B2,Out of Bag Data Length
0,HTR1F_GRCh37_3:88040823-88040823_Frame-Shift-D...,KIF2B_GRCh37_17:51902014-51902014_Frame-Shift-...,RAPGEF5_GRCh37_7:22197474-22197474_Frame-Shift...,C,C,C,NC,46
1,CENPF_GRCh37_1:214830653-214830653_Frame-Shift...,SERPINI1_GRCh37_3:167507159-167507159_Frame-Sh...,FARP1_GRCh37_13:99092237-99092237_Frame-Shift-...,C,C,C,NC,42
2,MIR1303_GRCh37_5:154065380-154065383_RNA_DEL_T...,MSANTD3_GRCh37_9:103204464-103204464_Frame-Shi...,SNAPC1_GRCh37_14:62242911-62242911_Frame-Shift...,C,C,C,NC,43
3,ZC3H18_GRCh37_16:88691141-88691141_Frame-Shift...,DAZAP1_GRCh37_19:1430254-1430254_Frame-Shift-D...,PTEN_GRCh37_10:89717770-89717770_Frame-Shift-D...,C,C,C,NC,41
4,MIR1303_GRCh37_5:154065380-154065383_RNA_DEL_T...,ARID5B_GRCh37_10:63850705-63850705_Frame-Shift...,CENPF_GRCh37_1:214830653-214830653_Frame-Shift...,C,C,C,NC,45
5,HTR1F_GRCh37_3:88040823-88040823_Frame-Shift-D...,KIF13A_GRCh37_6:17788023-17788023_Frame-Shift-...,SREK1IP1_GRCh37_5:64020298-64020298_Frame-Shif...,C,C,C,NC,37
6,MIR1303_GRCh37_5:154065380-154065383_RNA_DEL_T...,HTT_GRCh37_4:3133117-3133117_Frame-Shift-Del_D...,SREK1IP1_GRCh37_5:64020298-64020298_Frame-Shif...,C,C,C,NC,47
7,ZC3H18_GRCh37_16:88691141-88691141_Frame-Shift...,DAZAP1_GRCh37_19:1430254-1430254_Frame-Shift-D...,RAPGEF5_GRCh37_7:22197474-22197474_Frame-Shift...,C,C,C,NC,40
8,HTR1F_GRCh37_3:88040823-88040823_Frame-Shift-D...,KIF2B_GRCh37_17:51902014-51902014_Frame-Shift-...,ZC3H18_GRCh37_16:88691141-88691141_Frame-Shift...,C,C,C,NC,37
9,MIR1303_GRCh37_5:154065380-154065383_RNA_DEL_T...,HTT_GRCh37_4:3133117-3133117_Frame-Shift-Del_D...,ZC3H18_GRCh37_16:88691141-88691141_Frame-Shift...,C,C,C,NC,44


In [16]:
#find the average of the out of bag data length
average = treeChart['Out of Bag Data Length'].mean()
print('Average of the out of bag data length:', average)

#count the times that each mutation is selected as the top mutation, top left mutation, and top right mutation
topMutation = treeChart['Top Mutation'].value_counts()
topLeftMutation = treeChart['Top Left Mutation'].value_counts()
topRightMutation = treeChart['Top Right Mutation'].value_counts()

print('Top Mutation:', topMutation)
print('Top Left Mutation:', topLeftMutation)
print('Top Right Mutation:', topRightMutation)

Average of the out of bag data length: 40.8
Top Mutation: Top Mutation
MIR1303_GRCh37_5:154065380-154065383_RNA_DEL_TTTA-TTTA--           9
ZC3H18_GRCh37_16:88691141-88691141_Frame-Shift-Del_DEL_C-C--       5
HTR1F_GRCh37_3:88040823-88040823_Frame-Shift-Del_DEL_T-T--         4
SERPINI1_GRCh37_3:167507159-167507159_Frame-Shift-Del_DEL_A-A--    3
SNAPC1_GRCh37_14:62242911-62242911_Frame-Shift-Del_DEL_T-T--       2
CENPF_GRCh37_1:214830653-214830653_Frame-Shift-Del_DEL_A-A--       1
RAPGEF5_GRCh37_7:22197474-22197474_Frame-Shift-Del_DEL_T-T--       1
Name: count, dtype: int64
Top Left Mutation: Top Left Mutation
DAZAP1_GRCh37_19:1430254-1430254_Frame-Shift-Del_DEL_C-C--         4
HTT_GRCh37_4:3133117-3133117_Frame-Shift-Del_DEL_A-A--             3
CENPF_GRCh37_1:214830653-214830653_Frame-Shift-Del_DEL_A-A--       3
ARID5B_GRCh37_10:63850705-63850705_Frame-Shift-Del_DEL_A-A--       2
SERPINI1_GRCh37_3:167507159-167507159_Frame-Shift-Del_DEL_A-A--    2
KIF2B_GRCh37_17:51902014-51902014_Fram

In [17]:
def rand_forest_classify(sample, randomForest):
    numOfTrees = randomForest.shape[0]
    countC , countNC = 0, 0
    # for each tree in the random forest
    for i in range(numOfTrees):
        # if the sample has the top mutation
        if sample[randomForest['Top Mutation'][i]] == 1:
            # if the sample has the top left mutation
            if sample[randomForest['Top Left Mutation'][i]] == 1:
                # if the random forest classified the sample as cancerous
                if randomForest['A1'][i] == 'C':
                    countC += 1
                # if the random forest classified the sample as non-cancerous
                else:
                    countNC += 1
            # if the sample DOESNT have the top left mutation
            else:
                # if the random forest classified the sample as cancerous
                if randomForest['A2'][i] == 'C':
                    countC += 1
                # if the random forest classified the sample as non-cancerous
                else:
                    countNC += 1
        # if the sample DOESNT have the top mutation
        else:
            # if the sample has the top right mutation
            if sample[randomForest['Top Right Mutation'][i]] == 1:
                # if the random forest classified the sample as cancerous
                if randomForest['B1'][i] == 'C':
                    countC += 1
                # if the random forest classified the sample as non-cancerous
                else:
                    countNC += 1
            # if the sample DOESNT have the top right mutation
            else:
                # if the random forest classified the sample as cancerous
                if randomForest['B2'][i] == 'C':
                    countC += 1
                # if the random forest classified the sample as non-cancerous
                else:
                    countNC += 1
        
    if countC > countNC:
        return 'C', countC, countNC
    else:
        return 'NC', countC, countNC


In [18]:
# C1, C10, C15, NC5, and NC15

sample = data.loc['C1']
print('Sample: C1')
print('Classification:', rand_forest_classify(sample, treeChart))

sample = data.loc['C10']
print('Sample: C10')
print('Classification:', rand_forest_classify(sample, treeChart))

sample = data.loc['C15']
print('Sample: C15')
print('Classification:', rand_forest_classify(sample, treeChart))

sample = data.loc['NC5']
print('Sample: NC5')
print('Classification:', rand_forest_classify(sample, treeChart))

sample = data.loc['NC15']
print('Sample: NC15')
print('Classification:', rand_forest_classify(sample, treeChart))

Sample: C1
Classification: ('NC', 7, 18)
Sample: C10
Classification: ('NC', 5, 20)
Sample: C15
Classification: ('NC', 0, 25)
Sample: NC5
Classification: ('NC', 0, 25)
Sample: NC15
Classification: ('NC', 0, 25)


In [19]:
TP, TN, FP, FN = 0, 0, 0, 0

for i in range(outOfBagData.shape[0]):
    sample = outOfBagData.iloc[i]
    result, countC, countNC = rand_forest_classify(sample, treeChart)
    if result == 'C':
        if sample.name.startswith('C'):
            TP += 1
        else:
            FP += 1
    else:
        if sample.name.startswith('NC'):
            TN += 1
        else:
            FN += 1
 
print('TP:', TP)
print('TN:', TN)
print('FP:', FP)
print('FN:', FN)

print("len(outOfBagData):", len(outOfBagData))
print('Double check:', TP + TN + FP + FN)

advanced_metrics = calc_advanced_metrics(TP, FP, TN, FN)
advanced_metrics = pd.DataFrame(advanced_metrics, columns=['Accuracy', 'Sensitivity', 'Specificity', 'Precision', 'Miss Rate', 'FDR', 'FOR'])
advanced_metrics

# accuracy = total number of correct predictions / total number of predictions
# sensitivity = true positive rate = TP / (TP + FN) = ability to classify true positives from all predicted positives
# specificity = true negative rate = TN / (TN + FP) = ability to classify true negatives from all predicted negatives
# precision = TP / (TP + FP) = ability to classify true positives from all predicted positives
# miss rate = FN / (FN + TP) = ability to classify true negatives from all predicted negatives
# FDR = FP / (FP + TP) = ability to classify false postivies from all predicted positives
# FOR = FN / (FN + TN) = ability to classify false negatives from all predicted negatives


TP: 1
TN: 21
FP: 0
FN: 19
len(outOfBagData): 41
Double check: 41


,Accuracy,Sensitivity,Specificity,Precision,Miss Rate,FDR,FOR
0,53.66,5.00,100.00,100.00,95.00,0.00,47.50
